In [1]:
## This scripts serves as the extractor for all features used in this project
## it will iterate through each image in the dataset and extract features


In [2]:
# import modules
import pandas as pd
import numpy as np
import cv2 as cv
from radiomics import featureextractor
from IPython.display import clear_output


In [3]:
# Load training data csv. The original file contains the images names and patient data
training_data_csv = pd.read_csv('../merged_training_data.csv')
training_data_csv.head()

,image_id,age_approximate,sex,melanoma,seborrheic_keratosis
0,ISIC_0000000,55,female,0.0,0.0
1,ISIC_0000001,30,female,0.0,0.0
2,ISIC_0000002,60,female,1.0,0.0
3,ISIC_0000003,30,male,0.0,0.0
4,ISIC_0000004,80,male,1.0,0.0


In [4]:
## image and mask loading



def img_mask_load(img_name_load):
    
    ''' Function that takes the image name 
    from the Dataframe, searches this name in the image directory
    and loads it, along with the corresponding mask. Also, applies
    the mask to perform the segmentation along with histogram normalization
    
    
    
    '''
    
    im_path = '' ## CHANGE THIS VARIABLE WITH THE PATH WHERE YOU HAVE THE IMAGES
    mask_path = '' ## CHANGE THIS VARIABLE WITH THE PATH WHERE THE MAKS ARE
    
    img = cv.imread(im_path + img_name_load + '.jpg',-1)
    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    mask = cv.imread(img_name_load + 'segmentation.jpg', -1)
    img_segmented = cv.normalize(cv.bitwise_and(img, img, mask=mask),  output_array, 0, 255, cv.NORM_MINMAX)
    return img_segmented


In [5]:
## function for RGB features extraction


def features_rgb(img_name): 
    segmented = img_mask_load(img_name)
    
    # tuple with each color channel
    color = ('r','g','b')
    
    
    #  Declare list to store the most repeated pixel intensity in each channel
    mode_color_channel = []
    bp = []

# Iterate through each color channel and obtain histogram
    for i,col in enumerate(color):   
        histr = cv.calcHist([segmented],[i],None,[256],[1,256]) # the range should be [1,256] to exclude black background    
        mode_color_channel.append(list(histr.flatten()).index(max(histr.flatten()))) # append mode to mode list
        bp.append(segmented[:,:,i].flatten()[segmented[:,:,i].flatten().nonzero()])
        
        
    print(mode_color_channel, bp)
    return [mode_color_channel, bp].flatten()

In [6]:
## function for Radiomics features extraction

def features_radiomics(img_name):
    print('radiomics features')
    return None

In [7]:
## function to iterate through the images, extract the features and append them to the dataframe

def feature_extraction():
    
    for index, image in enumerate(training_data_csv['image_id']):
        feat_rgb = features_rgb(image)
        feat_radio = features_radiomics(image)
        
        print(feat_rgb)
        print(str(index+1) + '/' + str(len(training_data_csv['image_id']))+ ' images processed...')
        clear_output()
    return 

In [8]:
## Hit it! Extract the features

feature_extraction()

[ WARN:0@0.334] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('ISIC_0000000.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
